In [1]:
import numpy as np
import pandas as pd
import os
import json
import pickle
import random
import math
import re

In [2]:
def uniqueid():
    seed = random.getrandbits(64)
    while True:
        yield seed
        seed += 1

In [3]:
root_dir = "."
input_path = os.path.join(root_dir, 'raw_data', 'all_data_with_article_title_body.csv')
output_fpath = os.path.join(root_dir, 'processed', 'dictionary.pickle')

In [4]:
dictionary = []
url_to_label = {}

In [5]:
df = pd.read_csv(input_path)  # Load all the concatenated train, val, and test data

In [6]:
columns = ['url', 'title', 'body']
df[columns].head()

,url,title,body
0,http://www.politifact.com/truth-o-meter/statem...,"PolitiFact | Prior to Benghazi, were there 13 ...",As the U.S. House of Representatives was ready...
1,http://www.politifact.com/truth-o-meter/statem...,PolitiFact | Barack Obama said he&#39;s cut ta...,\nIn his speech at the Democratic National Com...
2,http://www.politifact.com/punditfact/statement...,PolitiFact | Juan Williams: No. 1 cause of dea...,"Amid ongoing protests in Ferguson, Mo., the na..."
3,http://www.snopes.com/donald-trumps-son-game-h...,Are These Donald Trump&#039;s Sons in Game-Hun...,NaN
4,http://www.politifact.com/truth-o-meter/statem...,PolitiFact | Hillary Clinton blames high-up Ru...,Editor's note: We've attached an update to thi...


In [7]:
df_final = df[columns]

In [8]:
df_final.shape

(55680, 3)

In [9]:
c = df_final.iloc[0]

In [10]:
c

url      http://www.politifact.com/truth-o-meter/statem...
title    PolitiFact | Prior to Benghazi, were there 13 ...
body     As the U.S. House of Representatives was ready...
Name: 0, dtype: object

In [11]:
#re.sub('[^A-Za-z0-9]+', '', mystring), use this if isalnum() doesn't work
def clean_up(s: str):
    if type(s) is float and math.isnan(s):
        return ""
    else:
        s = ''.join(e for e in s if e.isalnum() or e is ' ')
        s = re.sub("\s+"," ",s)
        return s.strip()

In [12]:
label_id_set = set()
label_id_record = {}
unique_sequence = uniqueid()
for idx in range(df_final.shape[0]):
    row = df_final.iloc[idx]
    record = {}
    
    if row['url'].strip() in url_to_label:
        continue
    
    label_id = str(next(unique_sequence))
    url_to_label[row['url']] = label_id
    record['cui'] = label_id
    
    record['title'] = clean_up(row['title'])
    record['description'] = clean_up(row['body'])
    record['type'] = ""
    label_id_set.add(label_id)
    dictionary.append(record)

In [13]:
assert len(dictionary) == len(label_id_set)

In [14]:
with open(output_fpath, 'wb') as write_handle:
    pickle.dump(dictionary, write_handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
with open("processed/url_cui_mapping.json", "w") as outfile:
    json.dump(url_to_label, outfile)